In [1]:
import zipfile
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
import torch
import torchvision
import torchvision.transforms as transforms
from operator import is_not
from functools import partial
from math import isclose

import time 

### load the data from files

In [2]:
df_events = pd.read_csv("events.csv")
df_game_info = pd.read_csv("ginf.csv")

### Creat training and testing labels number of goals in home and away match 

In [3]:
df_game_info_select=df_game_info[['fthg','ftag']].fillna(-1)
l1=np.array(df_game_info_select.values, dtype=int)
l1=l1
l1.shape

(10112, 2)

### one-hot encoding from labels 

In [4]:
coded_l1=np.empty((l1.shape[0],2))
for j in range (l1.shape[0]):
        coded_l1[j,0]=int(np.binary_repr(l1[j,0]))
        coded_l1[j,1]=int(np.binary_repr(l1[j,1]))
coded_l1.shape

(10112, 2)

### select features from events

In [5]:
df_events_select=df_events[['event_type','event_type2','shot_place','shot_outcome','location','side']]
#put NAn = -1
df_events_select=df_events_select.fillna(-1)
colums=['id_odsp','event_type','event_type2','shot_place','shot_outcome','location','side',]

### convert from pandas to numpy arry 

In [6]:
event_np=df_events_select.values
event_np=np.array(event_np,dtype=int)
event_np.shape

(941009, 6)

### pne-hot encoding events

In [7]:
coded_events=np.ones((event_np.shape[0],6))
for i in range (event_np.shape[0]):
    for j in range (event_np.shape[1]):
        coded_events[i,j]=int(np.binary_repr(event_np[i,j]))
coded_events.shape

(941009, 6)

### create event id to connect event-data with info-data

In [48]:
c1=df_events['id_odsp']
c1=(c1.values).reshape(-1,1)
c1.shape

(941009, 1)

In [49]:
full=np.concatenate((c1,event_np),axis=1)
full.shape

(941009, 7)

In [50]:
coded_full=np.concatenate((c1,coded_events),axis=1)
coded_full

array([['UFot0hit/', 1.0, 1100.0, ..., 10.0, 1001.0, 10.0],
       ['UFot0hit/', 10.0, -1.0, ..., -1.0, -1.0, 1.0],
       ['UFot0hit/', 10.0, -1.0, ..., -1.0, -1.0, 1.0],
       ...,
       ['z5L2OT5E/', 1.0, 1100.0, ..., 10.0, 1001.0, 10.0],
       ['z5L2OT5E/', 1000.0, -1.0, ..., -1.0, 100.0, 1.0],
       ['z5L2OT5E/', 11.0, -1.0, ..., -1.0, -1.0, 10.0]], dtype=object)

### divide events to home and away numpy arrries

In [11]:
full_home=[]
full_away=[]
for i in range (full.shape[0]):
    if full[i,6]==1:
        full_home.append(full[i])
    if full[i,6]==2:
        full_away.append(full[i])
full_home=np.array(full_home)[:,:6]
full_away=np.array(full_away)[:,:6]
print('full_home.shape',full_home.shape)
print('full_away.shape',full_away.shape)

full_home.shape (488224, 6)
full_away.shape (452785, 6)


### divide codedevents to home and away numpy arrries

In [12]:
coded_full_home=[]
coded_full_away=[]
for i in range (coded_full.shape[0]):
    if coded_full[i,6]==1:
        coded_full_home.append(coded_full[i])
    if coded_full[i,6]==10:
        coded_full_away.append(coded_full[i])
coded_full_home=np.array(coded_full_home)[:,:6]
coded_full_away=np.array(coded_full_away)[:,:6]
print('coded_full_home.shape',coded_full_home.shape)
print('coded_full_away.shape',coded_full_away.shape)

coded_full_home.shape (488224, 6)
coded_full_away.shape (452785, 6)


## create 3-d numpy array (match,events,events values) 

### create 3-d numpy array to home matches events

In [21]:
a3=-1*np.ones((9074,180,5),dtype=int) #180 maxumim number of events per match
j=0
c=0
for i in range (full_home.shape[0]-1):
    if full_home[i,0]==full_home[i+1,0]:
        a3[c,j]=full_home[i,1:]
        j+=1
    else:
        j=0
        c+=1
print('c',c)
full1=a3
full1.shape

c 9073


(9074, 180, 5)

### create 3-d numpy array to away matches events

In [22]:
a3=-1*np.ones((9074,180,5),dtype=int) #180 maxumim number of events per match
j=0
c1=0
for i in range (full_away.shape[0]-1):
    if full_away[i,0]==full_away[i+1,0]:
        a3[c1,j]=full_away[i,1:]
        j+=1
    else:
        j=0
        c1+=1
print('c1',c1)
full2=a3
full2.shape

c1 9073


(9074, 180, 5)

### create 3-d numpy array to  coded home matches events

In [23]:
a3=-1*np.ones((9074,180,5),dtype=int) #180 maxumim number of events per match
j=0
c=0
for i in range (coded_full_home.shape[0]-1):
    if coded_full_home[i,0]==coded_full_home[i+1,0]:
        a3[c,j]=coded_full_home[i,1:]
        j+=1
    else:
        j=0
        c+=1
print('c',c)
full3=a3
full3.shape

c 9073


(9074, 180, 5)

### create 3-d numpy array to  coded away matches events

In [24]:
a3=-1*np.ones((9074,180,5),dtype=int) #180 maxumim number of events per match
j=0
c=0
for i in range (coded_full_away.shape[0]-1):
    if coded_full_away[i,0]==coded_full_away[i+1,0]:
        a3[c,j]=coded_full_away[i,1:]
        j+=1
    else:
        j=0
        c+=1
print('c',c)
full4=a3
full4.shape

c 9073


(9074, 180, 5)

### vectorize data from (180*5) to (1,900)

In [28]:
training_data1=np.empty((9074,900))
for i in range (full1.shape[0]):
    training_data1[i]=full1[i].reshape(-1) 
training_data1.shape

(9074, 900)

In [29]:
training_data2=np.empty((9074,900))
for i in range (full2.shape[0]):
    training_data2[i]=full2[i].reshape(-1) 
training_data2.shape

(9074, 900)

In [30]:
training_data3=np.empty((9074,900))
for i in range (full3.shape[0]):
    training_data3[i]=full3[i].reshape(-1) 
training_data3.shape

(9074, 900)

In [31]:
training_data4=np.empty((9074,900))
for i in range (full4.shape[0]):
    training_data4[i]=full4[i].reshape(-1) 
training_data4.shape

(9074, 900)

# Part 2

# Training function using RNN 

In [32]:
import torch
import torch.nn as nn

# Set the random seed for reproducible results.
torch.manual_seed(1)

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        # This just calls the base class constructor.
        super().__init__()
        # Neural network layers assigned as attributes of a Module subclass
        # have their parameters registered for training automatically.
        self.rnn = torch.nn.RNN(input_size, hidden_size, nonlinearity='relu', batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The RNN also returns its hidden state but we don't use it.
        # While the RNN can also take a hidden state as input, the RNN
        # gets passed a hidden state initialized with zeros by default.
        x, _ = self.rnn(x)
        x = self.linear(x)
        return x
    
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [33]:
def train(model, train_data_gen,labels, criterion, optimizer):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # Set the model to training mode. This will turn on layers that would
    # otherwise behave differently during evaluation, such as dropout.
    model.train()
    # Store the number of sequences that were classified correctly.
    num_correct = 0
    
    # Iterate over every batch of sequences. Note that the length of a data generator
    # is defined as the number of batches required to produce a total of roughly 1000
    # sequences given a batch size.
    for batch_idx in range(len(train_data_gen)):
        # For each new batch, clear the gradient buffers of the optimized parameters.
        # Otherwise, gradients from the previous batch would be accumulated.
        optimizer.zero_grad()
        
        # Request a batch of sequences and class labels, convert them into tensors
        # of the correct type, and then send them to the appropriate device.
        data, target = train_data_gen[batch_idx],labels[batch_idx]
        data=data.view(1,1,900)
        target=target.view(1,1,1)
#         print('data.shape',data.shape)
#         print('target.shape',target.shape)
        data, target = data.float().to(device), target.long().to(device)
        
        # Perform the forward pass of the model.
        output = model(data)
        loss = criterion(output.float(), target.float())
#         print(loss)
        loss.backward()

        optimizer.step()
        
        y_pred = output
        if abs(y_pred - target.float()) <= 0.5:
            num_correct+=1
#         num_correct += (y_pred == target.float()).int().sum().item()

    return num_correct, loss.item()

In [62]:
def test(model, test_data_gen,labels_test, criterion):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # Set the model to evaluation mode. This will turn off layers that would
    # otherwise behave differently during training, such as dropout.
    model.eval()
    
    # Store the number of sequences that were classified correctly.
    num_correct = 0

    # A context manager is used to disable gradient calculations during inference
    # to reduce memory usage, as we typically don't need the gradients at this point.
    with torch.no_grad():
        for batch_idx in range(len(test_data_gen)):
            data, target = test_data_gen[batch_idx],labels_test[batch_idx]
#             print('test taget',target)
            data=data.view(1,1,900)
            target=target.view(1,1,1)
            
            data, target = (data).float().to(device), (target).long().to(device)
            
            output = model(data)

            loss = criterion(output.float(), target.float())
            y_pred = output
            if abs(y_pred - target.float()) <= 0.5:
                num_correct+=1
    return num_correct, loss.item()

In [81]:
def train_and_test(model, train_data_gen,labels, test_data_gen,labels_test, criterion, optimizer, max_epochs, verbose=True):
    # Automatically determine the device that PyTorch should use for computation.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    text_file_loss = open("loss_away_one_hot.txt", "w")
    text_file_acc = open("acc_away_one_hot.txt", "w")
    # Track the value of the loss function and model accuracy across epochs.
    history_train = {'loss': [], 'acc': []}
    history_test = {'loss': [], 'acc': []}
    
    for epoch in range(max_epochs):
        num_correct, loss = train(model, train_data_gen,labels, criterion, optimizer)
#         accuracy = float(num_correct) / (len(train_data_gen) * train_data_gen.batch_size) * 100
        accuracy = float(num_correct) / (train_data_gen.shape[0]) * 100
        history_train['loss'].append(loss)
        history_train['acc'].append(accuracy)
        
        # Do the same for the testing loop.
        num_correct, loss = test(model, test_data_gen,labels_test, criterion)
        accuracy = float(num_correct) / (len(test_data_gen) * test_data_gen) * 100
#         accuracy = float(num_correct) / (len(test_data_gen) * test_data_gen.numpy().batch_size) * 100
        history_test['loss'].append(loss)
        history_test['acc'].append(accuracy)

        if verbose or epoch + 1 == max_epochs:
            text_file_loss.write('{} , '.format(history_train['loss'][-1]))
            text_file_acc.write('{} , '.format(history_train['acc'][-1]))
            print('[Epoch {} / {}] loss: {}, acc {}%'.format(epoch + 1,max_epochs,history_train['loss'][-1],history_train['acc'][-1]))
    return model

## Train one-hot encoding data

### home without one-hot encoding

In [73]:

train_d=training_data1[0:2000]
# train_d=np,atrain_d,
train_d=torch.from_numpy(train_d)
print('train_d.shape',train_d.shape)
train_label=l1[0:2000,0].reshape(-1,1)
# np.concatenate((),axis=1)
train_label=torch.from_numpy(train_label)
print('train_label.shape',train_label.shape)
test_d=training_data1[2000:2100]
test_d=torch.from_numpy(test_d)
print('test_d.shape',test_d.shape)
test_label=l1[2000:2100,0]
test_label=torch.from_numpy(test_label)
print('test_label.shape',test_label.shape)

train_d.shape torch.Size([2000, 900])
train_label.shape torch.Size([2000, 1])
test_d.shape torch.Size([100, 900])
test_label.shape torch.Size([100])


In [74]:
# Setup the training and test data generators.
batch_size     = 100
train_data_gen = train_d
test_data_gen  = test_d
labels   =     train_label
labels_test=test_label
# print('train_data_gen.shape',train_data_gen.shape)
# print('labels.shape',labels.shape)
# Setup the RNN and training settings.
input_size  = 900
hidden_size = 500
output_size = 1
model       = SimpleRNN(input_size, hidden_size, output_size)
criterion   = torch.nn.MSELoss()
# optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer    =torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-3)
max_epochs  = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Train the model.
start=time.perf_counter()
model = train_and_test(model, train_data_gen,labels, test_data_gen,labels_test, criterion, optimizer, max_epochs)
# model=train(model, train_data_gen,labels, criterion, optimizer)
print(model)
print('Total time is {} Second'.format(time.perf_counter()-start))
#torch.save(model,'without_one_hot_encoding_last')

[Epoch 1 / 100] loss: 2.213989496231079, acc 25.8%
[Epoch 2 / 100] loss: 2.2713334560394287, acc 28.999999999999996%
[Epoch 3 / 100] loss: 1.7270112037658691, acc 30.0%
[Epoch 4 / 100] loss: 0.44491833448410034, acc 31.05%
[Epoch 5 / 100] loss: 1.29188072681427, acc 34.25%
[Epoch 6 / 100] loss: 0.2968190312385559, acc 33.800000000000004%
[Epoch 7 / 100] loss: 0.1385488212108612, acc 35.25%
[Epoch 8 / 100] loss: 0.32342100143432617, acc 35.449999999999996%
[Epoch 9 / 100] loss: 0.2702680230140686, acc 37.55%
[Epoch 10 / 100] loss: 0.47118890285491943, acc 38.85%
[Epoch 11 / 100] loss: 0.04121921584010124, acc 39.35%
[Epoch 12 / 100] loss: 0.12789642810821533, acc 40.699999999999996%
[Epoch 13 / 100] loss: 0.7619878053665161, acc 42.65%
[Epoch 14 / 100] loss: 1.2442492246627808, acc 45.6%
[Epoch 15 / 100] loss: 0.172923281788826, acc 46.5%
[Epoch 16 / 100] loss: 0.7130605578422546, acc 46.550000000000004%
[Epoch 17 / 100] loss: 0.2540840804576874, acc 46.25%
[Epoch 18 / 100] loss: 1.1361

### away without one-hot encoding

In [76]:

train_d=training_data2[0:2000]
# train_d=np,atrain_d,
train_d=torch.from_numpy(train_d)
print('train_d.shape',train_d.shape)
train_label=l1[0:2000,1].reshape(-1,1)
# np.concatenate((),axis=1)
train_label=torch.from_numpy(train_label)
print('train_label.shape',train_label.shape)
test_d=training_data2[2000:2100]
test_d=torch.from_numpy(test_d)
print('test_d.shape',test_d.shape)
test_label=l1[2000:2100,1]
test_label=torch.from_numpy(test_label)
print('test_label.shape',test_label.shape)

train_d.shape torch.Size([2000, 900])
train_label.shape torch.Size([2000, 1])
test_d.shape torch.Size([100, 900])
test_label.shape torch.Size([100])


In [77]:
# Setup the training and test data generators.
batch_size     = 100
train_data_gen = train_d
test_data_gen  = test_d
labels   =     train_label
labels_test=test_label
# print('train_data_gen.shape',train_data_gen.shape)
# print('labels.shape',labels.shape)
# Setup the RNN and training settings.
input_size  = 900
hidden_size = 500
output_size = 1
model       = SimpleRNN(input_size, hidden_size, output_size)
criterion   = torch.nn.MSELoss()
# optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer    =torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-3)
max_epochs  = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Train the model.
start=time.perf_counter()
model = train_and_test(model, train_data_gen,labels, test_data_gen,labels_test, criterion, optimizer, max_epochs)
# model=train(model, train_data_gen,labels, criterion, optimizer)
print(model)
print('Total time is {} Second'.format(time.perf_counter()-start))
#torch.save(model,'without_one_hot_encoding_last')

[Epoch 1 / 100] loss: 0.22605489194393158, acc 31.25%
[Epoch 2 / 100] loss: 0.007035145536065102, acc 32.15%
[Epoch 3 / 100] loss: 0.0006912393146194518, acc 33.2%
[Epoch 4 / 100] loss: 0.003385464195162058, acc 34.8%
[Epoch 5 / 100] loss: 0.37672752141952515, acc 37.15%
[Epoch 6 / 100] loss: 0.005100299138575792, acc 35.35%
[Epoch 7 / 100] loss: 0.36848849058151245, acc 37.5%
[Epoch 8 / 100] loss: 0.016117004677653313, acc 39.45%
[Epoch 9 / 100] loss: 0.004214274697005749, acc 39.75%
[Epoch 10 / 100] loss: 4.060683750140015e-06, acc 39.900000000000006%
[Epoch 11 / 100] loss: 0.02343786135315895, acc 40.849999999999994%
[Epoch 12 / 100] loss: 0.1439531445503235, acc 42.15%
[Epoch 13 / 100] loss: 0.029528601095080376, acc 42.35%
[Epoch 14 / 100] loss: 0.028509659692645073, acc 42.199999999999996%
[Epoch 15 / 100] loss: 0.08163485676050186, acc 44.0%
[Epoch 16 / 100] loss: 0.03109005093574524, acc 45.300000000000004%
[Epoch 17 / 100] loss: 0.0030980631709098816, acc 47.55%
[Epoch 18 / 10

### home one-hot encoding

In [79]:

train_d=training_data3[0:2000]
# train_d=np,atrain_d,
train_d=torch.from_numpy(train_d)
print('train_d.shape',train_d.shape)
train_label=coded_l1[0:2000,0].reshape(-1,1)
# np.concatenate((),axis=1)
train_label=torch.from_numpy(train_label)
print('train_label.shape',train_label.shape)
test_d=training_data3[2000:2100]
test_d=torch.from_numpy(test_d)
print('test_d.shape',test_d.shape)
test_label=coded_l1[2000:2100,0]
test_label=torch.from_numpy(test_label)
print('test_label.shape',test_label.shape)

train_d.shape torch.Size([2000, 900])
train_label.shape torch.Size([2000, 1])
test_d.shape torch.Size([100, 900])
test_label.shape torch.Size([100])


In [80]:
# Setup the training and test data generators.
batch_size     = 100
train_data_gen = train_d
test_data_gen  = test_d
labels   =     train_label
labels_test=test_label
# print('train_data_gen.shape',train_data_gen.shape)
# print('labels.shape',labels.shape)
# Setup the RNN and training settings.
input_size  = 900
hidden_size = 500
output_size = 1
model       = SimpleRNN(input_size, hidden_size, output_size)
criterion   = torch.nn.MSELoss()
# optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer    =torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-3)
max_epochs  = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Train the model.
start=time.perf_counter()
model = train_and_test(model, train_data_gen,labels, test_data_gen,labels_test, criterion, optimizer, max_epochs)
# model=train(model, train_data_gen,labels, criterion, optimizer)
print(model)
print('Total time is {} Second'.format(time.perf_counter()-start))
#torch.save(model,'without_one_hot_encoding_last')

[Epoch 1 / 100] loss: 48.84050750732422, acc 2.55%
[Epoch 2 / 100] loss: 49.902740478515625, acc 3.4000000000000004%
[Epoch 3 / 100] loss: 102.30608367919922, acc 4.3999999999999995%
[Epoch 4 / 100] loss: 21.406017303466797, acc 7.249999999999999%
[Epoch 5 / 100] loss: 44.58956527709961, acc 11.85%
[Epoch 6 / 100] loss: 16.115751266479492, acc 15.5%
[Epoch 7 / 100] loss: 1.1055079698562622, acc 20.25%
[Epoch 8 / 100] loss: 1.7203938961029053, acc 24.0%
[Epoch 9 / 100] loss: 2.6252493858337402, acc 16.400000000000002%
[Epoch 10 / 100] loss: 3.5374975204467773, acc 0.95%
[Epoch 11 / 100] loss: 4.440035343170166, acc 1.0%
[Epoch 12 / 100] loss: 5.1578216552734375, acc 0.6%
[Epoch 13 / 100] loss: 6.076781749725342, acc 1.05%
[Epoch 14 / 100] loss: 6.703622341156006, acc 1.9%
[Epoch 15 / 100] loss: 7.669404029846191, acc 1.55%
[Epoch 16 / 100] loss: 8.055832862854004, acc 1.0%
[Epoch 17 / 100] loss: 8.808653831481934, acc 0.95%
[Epoch 18 / 100] loss: 9.318214416503906, acc 0.95%
[Epoch 19 /

### Away one-hot encoding

In [82]:

train_d=training_data4[0:2000]
# train_d=np,atrain_d,
train_d=torch.from_numpy(train_d)
print('train_d.shape',train_d.shape)
train_label=coded_l1[0:2000,1].reshape(-1,1)
# np.concatenate((),axis=1)
train_label=torch.from_numpy(train_label)
print('train_label.shape',train_label.shape)
test_d=training_data4[2000:2100]
test_d=torch.from_numpy(test_d)
print('test_d.shape',test_d.shape)
test_label=coded_l1[2000:2100,1]
test_label=torch.from_numpy(test_label)
print('test_label.shape',test_label.shape)

train_d.shape torch.Size([2000, 900])
train_label.shape torch.Size([2000, 1])
test_d.shape torch.Size([100, 900])
test_label.shape torch.Size([100])


In [83]:
# Setup the training and test data generators.
batch_size     = 100
train_data_gen = train_d
test_data_gen  = test_d
labels   =     train_label
labels_test=test_label
# print('train_data_gen.shape',train_data_gen.shape)
# print('labels.shape',labels.shape)
# Setup the RNN and training settings.
input_size  = 900
hidden_size = 500
output_size = 1
model       = SimpleRNN(input_size, hidden_size, output_size)
criterion   = torch.nn.MSELoss()
# optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer    =torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-3)
max_epochs  = 100
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Train the model.
start=time.perf_counter()
model = train_and_test(model, train_data_gen,labels, test_data_gen,labels_test, criterion, optimizer, max_epochs)
# model=train(model, train_data_gen,labels, criterion, optimizer)
print(model)
print('Total time is {} Second'.format(time.perf_counter()-start))
#torch.save(model,'without_one_hot_encoding_last')

[Epoch 1 / 100] loss: 100.2083740234375, acc 2.4%
[Epoch 2 / 100] loss: 104.42540740966797, acc 4.7%
[Epoch 3 / 100] loss: 15.651537895202637, acc 10.0%
[Epoch 4 / 100] loss: 54.869449615478516, acc 15.15%
[Epoch 5 / 100] loss: 0.1483779102563858, acc 18.5%
[Epoch 6 / 100] loss: 0.03042624518275261, acc 24.099999999999998%
[Epoch 7 / 100] loss: 0.00032884188112802804, acc 28.199999999999996%
[Epoch 8 / 100] loss: 21.20081329345703, acc 31.2%
[Epoch 9 / 100] loss: 0.3162119686603546, acc 22.45%
[Epoch 10 / 100] loss: 0.7997596859931946, acc 1.3%
[Epoch 11 / 100] loss: 798.501708984375, acc 1.6%
[Epoch 12 / 100] loss: 586.9741821289062, acc 2.25%
[Epoch 13 / 100] loss: 89.3946762084961, acc 1.2%
[Epoch 14 / 100] loss: 2.426917791366577, acc 1.95%
[Epoch 15 / 100] loss: 1.4314595460891724, acc 1.7000000000000002%
[Epoch 16 / 100] loss: 18.80291748046875, acc 0.8500000000000001%
[Epoch 17 / 100] loss: 15.917948722839355, acc 1.15%
[Epoch 18 / 100] loss: 12.829432487487793, acc 0.5499999999